In [5]:
import openai

In [6]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_KEY')

In [7]:
openai.api_key = OPENAI_API_KEY

In [8]:
def get_response(messages,model="gpt-4o-mini"):

    response = openai.chat.completions.create(
        model=model,
        messages=messages)

    return response.choices[0].message.content

In [10]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [11]:
from datasets import load_dataset

In [15]:
dataset = load_dataset("bigbio/med_qa", name="med_qa_en_source", split="train")

data_clean.zip:   0%|          | 0.00/132M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/dev.jsonl


In [18]:
print(dataset['question'][1])

A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?


In [19]:
import pandas as pd
df = pd.DataFrame(dataset)
print(df.head())

  meta_info                                           question answer_idx  \
0   step2&3  A 23-year-old pregnant woman at 22 weeks gesta...          E   
1   step2&3  A 3-month-old baby died suddenly at night whil...          A   
2     step1  A mother brings her 3-week-old infant to the p...          A   
3     step1  A pulmonary autopsy specimen from a 58-year-ol...          A   
4     step1  A 20-year-old woman presents with menorrhagia ...          E   

                                              answer  \
0                                     Nitrofurantoin   
1  Placing the infant in a supine position on a f...   
2       Abnormal migration of ventral pancreatic bud   
3                                    Thromboembolism   
4                             Von Willebrand disease   

                                             options  
0  [{'key': 'A', 'value': 'Ampicillin'}, {'key': ...  
1  [{'key': 'A', 'value': 'Placing the infant in ...  
2  [{'key': 'A', 'value': 'Abnormal

In [30]:
options=[]
for i in range(5):
  strin=''
  for val in df['options'][i]:
    strin= strin + val['key'] + ') ' + val['value'] + '\n'
  options.append(strin)
print(options[0])

A) Ampicillin
B) Ceftriaxone
C) Ciprofloxacin
D) Doxycycline
E) Nitrofurantoin



In [32]:
user_prompts=[]
for i in range(5):
  user_prompts.append(df['question'][i] + '\n' + options[i])
print(user_prompts[0])



A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
A) Ampicillin
B) Ceftriaxone
C) Ciprofloxacin
D) Doxycycline
E) Nitrofurantoin



In [33]:
first_prompt = user_prompts[0]
first_prompt_answer = df['answer'][0]
second_prompt = user_prompts[1]
second_prompt_answer = df['answer'][1]
third_prompt = user_prompts[2]
third_prompt_answer = df['answer'][2]
fourth_prompt = user_prompts[3]

In [34]:
system_role='You are a medical student who has been trained to answer questions based on medical science. You will be asked mutiple choice questions and you have to return the right answer.'

### **Few Shot Prompting**

In [35]:
prompt=f'''Question: {first_prompt}
Answer: {first_prompt_answer}
Question: {second_prompt}
Answer: {second_prompt_answer}
Question: {third_prompt}
Answer: {third_prompt_answer}
Question: {fourth_prompt}
'''

In [36]:
messages=[{'role':'system','content':system_role},{'role':'user','content':prompt}]

In [37]:
print(get_response(messages))

Answer: Thromboembolism


In [38]:
df['answer'][3]

'Thromboembolism'

Correct answer provided by GPT 4o mini

In [39]:
print(get_response(messages=messages, model='gpt-4o'))

Answer: Thromboembolism


Correct Answer by GPT 4o

**One Shot Prompting**

In [40]:
fifth_question = user_prompts[4]

prompt=f'''Question: {first_prompt}
Answer: {first_prompt_answer}
Question: {fifth_question}'''

In [42]:
messages=[{'role':'system','content':system_role},{'role':'user','content':prompt}]

In [44]:
print(get_response(messages))

The patient's symptoms and laboratory findings suggest a bleeding disorder. She has menorrhagia, easy bruising, a normal platelet count, a normal PT, and an elevated PTT. These findings are consistent with von Willebrand disease, which is the most common inherited bleeding disorder and can lead to both menorrhagia and easy bruising. 

Answer: E) Von Willebrand disease


In [45]:
# Checking the right answer
df['answer'][4]

'Von Willebrand disease'

**Zero shot prompting**

In [46]:
prompt=f'''Question: {first_prompt}
Answer:'''

In [47]:
messages=[{'role':'system','content':system_role},{'role':'user','content':prompt}]

In [48]:
print(get_response(messages))

The clinical presentation of this pregnant woman suggests a urinary tract infection (UTI), characterized by burning upon urination. Given her pregnancy status, the choice of antibiotic must be safe for both the mother and the fetus.

Among the options provided, Nitrofurantoin (E) is considered a first-line treatment for uncomplicated UTIs in pregnant women, particularly in the early second trimester. It is generally safe to use during pregnancy, especially before 36 weeks of gestation.

Thus, the best treatment for this patient is **E) Nitrofurantoin**.


In [49]:
df['answer'][0]

'Nitrofurantoin'